In [3]:
%%HTML
<style>
div.prompt.input_prompt {display:none}
</style>

# Introduction

Autoencoders are neural networks that seek to encode data into a latent-space representation, and decode this data to obtain emphasized features. Being conceptually easy to explain, they are notorious for being one of the first networks that new students learn. Its appeal stems from its idealistic simplicity. 

Autoencoders have had various applications over the years, ranging from dimensionality reduction to denoising image datasets. Our aim today, is to test its most traditional function as an encoder and test the hypothesis: 

**"Can an Autoencoder + Convolutional Neural Network approach perform better than a simple Convolutional Neural Network classifier on a classification task?"**

The way this experiment will be approached is to train an encoder, use the layers up till the latent space representation as input for the Convolutional Neural Network, and determine whether it outperforms a simple Convolutional Neural Network approach and in what ways. We will be looking at the performance of the Autoencoder, select CNN models, and the Autoencoder + those CNN models on the CIFAR-10 dataset. 

In particular, we will demonstrate how adding the autoencoder to a shallow network such as the aptly-named ShallowNet can increase classification accuracy, but when applied to deeper networks such as MiniVGGNet, can actually reduce classification accuracy compared to more classical approaches. 

# Dataset Explanation 

For our experiment, we are going to employ a benchmark dataset, CIFAR-10. 

There are a few notable qualities to CIFAR-10:

First, it has 50,000 training images, and 10,000 testing images, with an even split between 10 classes each, giving us 5,000 training images per class and 1,000 testing images per class. This is a smaller dataset, especially in comparison to datasets that have millions of images like ImageNet. It has an even split between classes to ensure that no one class is trained significantly more than the other.

It should be noted that for this particular experiment, we have the constraint to use only 50% of training data for the following three classes: bird, deer, and truck. Other classes maintain their full amount of training data.

Next, the images themselves are 32 x 32 with 3 channels (RGB). This is a very small amount of data for each image, which makes it difficult to get high accuracy.

Finally, the images, depending on the class, can feature a significant amount of blur, deformation, occlusion, viewpoint variation, and intra-class variation. This not only makes it difficult to gain high accuracy, but the generalizability of the model also suffers in question. This means we need both robust architecture and the application of regularization techniques to assist in generalization.

# Model/Architecture Explanation

We will be evaluating the classification task on the CIFAR-10 dataset with two models: ShallowNet and MiniVGGNet [1]. Our choice of models allows us to evaluate the effectiveness of the autoencoder on the simplest of CNNs possible (ShallowNet) as a baseline, then evaluate it on a model with more complexity (MiniVGGNet). Both of these networks will be evaluated on performance with and without the encoder as input.
## ShallowNet

![title](./Images/ShallowNet.png)

As mentioned above, ShallowNet contains just 1 CONV and FC layer each, being the simplest implementation of a CNN possible. By using ShallowNet, we can test the baseline hypothesis that autoencoders are able to provide useful information and depth to very shallow algorithms, establishing that the latent-space representation can have value to a CNN. 
## MiniVGGNet

![image.png](./Images/MiniVGGNet.png)

As VGGNet is usually evaluated on ImageNet instead of CIFAR-10 and is traditionally either 16 or 19 layers, we will be reducing the size of the model as seen in Table 2 to work with our smaller dataset [1]. The reasoning behind choosing MiniVGGNet over another similar depth network is because it employs multiple CONV => RELU layers before the POOL layer, and doing so allows the model to glean richer features. If the performance of the network still improves with the autoencoder, this indicates the latent-space representation contains a great amount of relevant information for the convolutional layers to use in discerning features. 

The batch normalization and the dropout used in MiniVGGNet serve to combat overfitting and improve generalizability of our model on the validation data. 

## Autoencoder

![title](./Images/Convolutional_Autoencoder.png)

The autoencoder we are going to use for our model is a purely convolutional model (no fully connected layers) that employs shallow, strided convolutions in place of max pooling, with batch normalization [2]. It has been demonstrated to accurately replicate images [2] from CIFAR-10 with no noticeable differences between the actual image data and the representations from the autoencoder. We will train this model separately, take the trained encoder and apply it to each of the aforementioned models.

Fully connected layers were found to reduce image quality [2]. Personal experiments were made to compress the data down to 8 x 8 and 4 x 4 via strided convolutions in an attempt improve classification accuracy. However, not only did the classification accuracy decrease, this resulted in the loss of too much detail given the low resolution of the CIFAR-10 images, so the compression was kept to a minimum of 16 x 16 as per the author's recommendation. Sigmoid was used as the activation function in the last layer to produce output in the range of [0,1]. 

## Autoencoder + ShallowNet, Autoencoder + MiniVGGNet

![title](./Images/Autoencoder+CNN.png)

For our autoencoder + CNN models, we will take the encoding portion of the autoencoder model (the layers up until just before the UPSAMPLING layer) and use that as input for our CNN. Note that since the encoder does not heavily compress the data, it is not a necessity to will not need to add any upsampling or convolutional transpose layers.

# Experiment

For our experiment, we will be testing how well the network classifies image labels from CIFAR-10. We will use 50% of the training data from the bird, deer and truck classes and 100% of the training data from the rest of the classes in CIFAR-10. 

## Hyperparameters

For ShallowNet, data augmentation was employed with rotation up to 10 degrees, slight width and height shift range, and horizontal flip applied randomly. The epochs and the batch size were set to 40 and 32, respectively. For our optimization algorithm, since we are not expecting any troubles converging quickly, SGD was used with a learning rate of 0.01 and no additional parameters. The simplicity of the hyperparameters reflect the simplicity and depth of the model here.

For MiniVGGNet, we employed the same data augmentation as ShallowNet, with rotation up to 10 degrees, slight width and height shift range, and horizontal flip applied randomly. 150 epochs were used with batch size of 64, as the model needed an extensive amount of epochs to converge. SGD was used as the optimization algorithm of choice. We have kept the learning rate set to 0.01 to for a balance between convergence speed and accuracy, with higher values adding unwanted variation to the validation loss. The decay was set to 0.01/200, with the momentum = 0.9 and the nesterov acceleration set on to aid in convergence. 

For the autoencoder, 5 epochs [2] was found to be sufficient in ensuring convergence. 
Given the low number of epochs, Adam was used as our optimization with a learning rate of 0.001 to support fast convergence. In addition, a batch size of 32 was sufficient to train the network. 

For our Autoencoder + (ShallowNet, MiniVGGNet) models, the same hyperparameters were employed as the base models. The only change present is the increase of epochs to 200 for the Autoencoder + MiniVGGNet model. Despite experimentation around the learning rate to attempt to make convergence faster, the best results were still found with 200 epochs and a SGD learning rate of 0.01.

## Evaluation Metrics

For the CNN models, our primary evaluation metrics will be the confusion matrices (actual vs predicted values for each class of CIFAR-10), validation accuracy, precision, recall, and f-score for each model.

For the autoencoders, we'll be comparing the raw image data to the image representations created from the autoencoder and comparing the similarity visually. 

For both models, we'll also be visualizing the training versus validation loss in order to gauge whether the network is overfitting and ensure reasonable convergence.

# Results

### Autoencoder
<table><tr><td><img src="./Models/output/plots/convautoencoder/autoencoder_only_plot.png" height="1300" width="600"></td><td><img src="./Models/output/plots/convautoencoder/autoencoder_only_output.png" width="100"/></td><tr></table><table><tr><td><img src="./Images/autoencoder_training_and_validation_loss_caption.png"/></td><td><img src="./Images/autoencoder_image_replication_caption.png"></td><tr></table>

With our autoencoder, we have quickly reached a nice low plateau for our validation loss. Upon inspection, it is hard to spot any differences between the raw image data on the left and the image representations on the right. Therefore, the model is performing well. 

### ShallowNet
<table><tr><td><img src="./Models/output/plots/shallownet/cifar10_shallownet_conf_matrix.png" width="800"/></td><tr><table><table><tr><td><img src="./Images/shallownet_conf_matrix_caption.png" width="320"/></td><tr><table><table><tr><td><img src="./Models/output/plots/shallownet/cifar10_shallownet.png" width="400"/></td><td><img src="./Models/output/plots/shallownet/cifar10_shallownet_classification_report.png" height="1300" width="700"></td><tr></table><table><tr><td><img src="./Images/shallownet_training_and_validation_loss_caption.png"/></td><td><img src="./Images/shallownet_classification_report_caption.png"></td><tr></table>

The validation loss has nicely plateaud at 40 epochs, indicating that the networks has converged and any further training will lead to overfitting.

Given the precision and recall values of ships, trucks, and automobiles, it is safe to say that the network has not learned enough features to be able to effectively distinguish beyond automobiles to other, more specific vehicle types (trucks and ships). 

In addition, the network is having difficulty precisely identifying between different small animals, with birds, deer, and dogs often mistaken to be frogs (which have low precision and high recall) and dogs often mistaken for cats. Again, the model has not learnt enough features to distinguish past frogs to other small animals.

In theory, we should hope to expect improvements to the aforementioned with an autoencoder input. Let's see if this is the case.

### Autoencoder + ShallowNet

<table><tr><td><img src="./Models/output/plots/convautoencoder_shallownet/cifar10_convautoencoder_shallownet_conf_matrix.png" width="800"/></td><tr><table><table><tr><td><img src="./Images/autoencoder_shallownet_conf_matrix_caption.png" width="450"/></td><tr><table><table><tr><td><img src="./Models/output/plots/convautoencoder_shallownet/cifar10_convautoencoder_shallownet.png" width="400"/></td><td><img src="./Models/output/plots/convautoencoder_shallownet/cifar10_convautoencoder_shallownet_classification_report.png" height="1300" width="700"></td><tr></table><table><tr><td><img src="./Images/autoencoder_shallownet_training_and_validation_loss_caption.png"/></td><td><img src="./Images/autoencoder_shallownet_classification_report_caption.png"></td><tr></table>

Once again, the validation loss has nicely plateaud at 40 epochs, indicating that the networks has converged. However, there is slight overfitting and any further training will exacerbate this.

Comparing the vehicle evaluation metrics of this model to the ShallowNet, we see that the algorithm has learned features that improve its ability to move past automobiles and correctly identify between ships, airplanes, and trucks. While this has resulted in a reduction of recall of the automobile class, it makes up for that with the increase in recall for the other vehicles.

We also observe nominal improvements in the precision of the frog class and in the recall of the other small animal classes (excluding dog), indicating that the model has learned a few more specialized features to be able to better distinguish from frogs and other small animals.

However, we also observe a very notable spike in false positives for the cat class, especially with dogs. So it is presumable that there has been a loss of information that used to distinguish these other animals from cats.

While the accuracy is not terribly high, it is clear that the latent space representation provides this model with more polished features that can disinguish between classes in certain groups (vehicles and small animals).

### MiniVGGNet
<table><tr><td><img src="./Models/output/plots/minivggnet/cifar10_minivggnet_conf_matrix.png" width="800"/></td><tr><table><table><tr><td><img src="./Images/minivggnet_conf_matrix_caption.png" width="320"/></td><tr><table><table><tr><td><img src="./Models/output/plots/minivggnet/cifar10_minivggnet.png" width="400"/></td><td><img src="./Models/output/plots/minivggnet/cifar10_minivggnet_classification_report.png" height="1300" width="700"></td><tr></table><table><tr><td><img src="./Images/minivggnet_training_and_validation_loss_caption.png"/></td><td><img src="./Images/minivggnet_classification_report_caption.png"></td><tr></table>

For this model, we see convergence of the validation accuracy and stabilization of the validation loss, with only very slight overfitting present.

Unlike ShallowNet, our network has enough depth to distinguish between vehicle type quite well, with only 7.3% of trucks being mistakenly classified as automobiles, compared to ~33% from Figure 1.

The problem of high recall + low precision with frogs persists, wherein small animals are regularily misclassified as frogs instead of bird, cat, dog, or deer. However this is been greatly diminished compared to the base ShallowNet, demonstrating that the depth of the network is conducive to obtaining meaningful features. 

Let's see if the autoencoder assists our network by passing it even more relevant information as input.

### Autoencoder + MiniVGGNet
<table><tr><td><img src="./Models/output/plots/convautoencoder_minivggnet/cifar10_convautoencoder_minivggnet_conf_matrix.png" width="800"/></td><tr><table><table><tr><td><img src="./Images/autoencoder_minivggnet_conf_matrix_caption.png" width="450"/></td><tr><table><table><tr><td><img src="./Models/output/plots/convautoencoder_minivggnet/cifar10_convautoencoder_minivggnet.png" width="400"/></td><td><img src="./Models/output/plots/convautoencoder_minivggnet/cifar10_convautoencoder_minivggnet_classification_report.png" height="1300" width="700"></td><tr></table><table><tr><td><img src="./Images/autoencoder_minivggnet_training_and_validation_loss_caption.png"/></td><td><img src="./Images/autoencoder_minivggnet_classification_report_caption.png"></td><tr></table>

Compared to our MiniVGGNet approach, there is a drop in average accuracy, precision, recall, and f-score. Our model is definitely performing *worse* than before applying an autoencoder....but where is it exactly experiencing this trouble? Let's find out.

While the network lost a small percentage of its ability to accurately determine trucks from automobiles, it is not a siginificant enough amount to explain the difference in performance. Let's look elsewhere.

Where we see our major differences is again in the frog and cat classes' precision (0.59 and 0.65 in Table 8 versus 0.69 and 0.78 in Table 7, respectively), where other small animals are regularily misclassified as frogs and cats more often. 
The autoencoder seems to have compressed against information that made these distinguishments possible. Perhaps edges around the ears, eyes, head or some other critical position that were once more defined are now dulled.

In sum, the autoencoder + CNN approach seems to have lost some relevant information useful for networks of this depth and structure.

## Overall
![title](./Images/model_comparison_classification_report.png)

For ShallowNet, we experienced improvements in average classification accuracy (0.59 to 0.64), precision (0.62 to 0.65), recall (0.59 to 0.64), and f1-score (0.57 to 0.62) when the autoencoder was applied. Most of these improvements were from increases in accuracy on bird, cat, deer, ship, and truck classes.

For MiniVGGNet, we observed decreases across the board in average classification accuracy (0.84 to 0.8), precision (0.84 to 0.82), recall (0.84 to 0.8) and f1-score (0.84 to 0.8). While nearly all classes experienced accuracy drops, the animal classes in particular seemed to suffer.

# Conclusion

In regards to our initial hypothesis of "Can an Autoencoder + Convolutional Neural Network approach perform better than a simple Convolutional Neural Network classifier on a classification task?", we can conclude that for the shallow network selected, an autoencoder increases accuracy and for the slightly deeper network selected, an autoencoder lowers accuracy.

These differences were most noticeable in the animal classes for both datasets, where the correct classification of birds, dogs, cats and deer was what changed the most (positively or negatively) when autoencoders were applied. 

It is speculated that this is because shallow networks do not have the depth to extract enough information about features, which autoencoders help to provide. On the other hand, established deeper architectures have specific methods to extract relevant features, and adding an autoencoder beforehand may lose depth of data through compression. Further exploration is necessary.

# Further Exploration

Our analysis was limited by the type, depth and structure of networks we explored and the conclusions we can take from them are just as limited. These are some ideas for future exploration of this topic. 

- Explore other more complex networks such as ResNet
- Explore how an autoencoder performs with SqueezeNet (to determine whether the small model size will synergize)
- Limiting the amount of training data for other classes to 50% all across the board, or playing with different ratios
- Adding in fully connected layers, and seeing if the image representation getting worse actually correlates with worse peformance in models
- Training the autoencoder with the CNN model, instead of pretraining and freezing the layers beforehand
- Upsampling the latent space representation of the image to restore to the original 32 x 32 dimensions

# Bibliography

[1] A. Rosebrock, Deep Learning for Computer Vision, PyImageSearch, https://www.pyimagesearch.com/deep-learning-computer-vision-python-book/, accessed on 24 March 2020, pp 208-210, 231-241. <br>
[2] R. Flynn, Convolutional Autoencoders for the Cifar10 Dataset, https://github.com/rtflynn/Cifar-Autoencoder, accessed on 24 March 2020.